In [1]:
import os
import sys
import zarr
import tensorstore as ts
from numcodecs import Blosc
import ome_zarr
from ome_zarr.io import parse_url
from ome_zarr.writer import write_image
import dask.array as da
from dask import delayed
import napari
import numpy as np
from tqdm.notebook import tqdm 

from sqlalchemy import create_engine
from sqlalchemy.orm import sessionmaker

sys.path.append('..')
from tracks_interactions.db.db_model import CellDB, TrackDB

In [2]:
os.environ['NAPARI_PERFMON'] = '0'

from napari.settings import get_settings
settings = get_settings()
settings.experimental.async_

False

In [3]:
# get images
ch0_path = r'D:\kasia\tracking\E6_exp\E6_C0.zarr'
ch1_path = r'D:\kasia\tracking\E6_exp\E6_C1.zarr'

ch0_list = []
for level in range(1,5):
    ch0_list.append(da.from_zarr(ch0_path,level))

ch1_list = []
for level in range(1,5):
    ch1_list.append(da.from_zarr(ch1_path,level))

In [4]:
# get access to the database
new_db_path = r'D:\kasia\tracking\E6_exp\double_segmentation_ultrack\Exp6_gardener_v6.db'
#r'C:\Users\kmk280\Exp6_gardener_v6.db'
engine = create_engine(f'sqlite:///{new_db_path}')
session = sessionmaker(bind=engine)()

In [5]:
from sqlalchemy import text
session.execute(text("ANALYZE;"))
session.commit()

In [6]:
new_db_path

'D:\\kasia\\tracking\\E6_exp\\double_segmentation_ultrack\\Exp6_gardener_v6.db'

In [7]:
def build_frame():

    global viewer1

    if viewer1.layers['Labels'].visible:

        query_lim = 500

        # get the corner pixels of the viewer - for magnification
        corner_pixels = viewer1.layers['Labels'].corner_pixels

        r_rad = (corner_pixels[1, 0] - corner_pixels[0, 0])/2
        c_rad = (corner_pixels[1, 1] - corner_pixels[0, 1])/2

        # get the center position of the viewer
        r = viewer1.camera.center[1]
        c = viewer1.camera.center[2]

        # calculate labels extent
        r_start = r - r_rad
        r_stop = r + r_rad
        c_start = c - c_rad
        c_stop = c + c_rad 

        ind = viewer1.dims.current_step[0]

        viewer1.layers['Labels'].data[:] = 0

        query = session.query(CellDB).filter(CellDB.t == ind).filter(CellDB.bbox_0 < int(r_stop)).filter(CellDB.bbox_1 < int(c_stop)).filter(CellDB.bbox_2 > int(r_start)).filter(CellDB.bbox_3 > int(c_start)).limit(query_lim).all()
    

        if len(query) < query_lim:

            frame = viewer1.layers['Labels'].data

            for cell in query:

                frame[cell.bbox_0:cell.bbox_2, cell.bbox_1:cell.bbox_3] += (cell.mask.astype(int) * cell.track_id)

            viewer1.layers['Labels'].data = frame
            viewer1.status = f'Found {len(query)} cells in the field.'

        else:

            viewer1.layers['Labels'].refresh()
            viewer1.status = f'More than {query_lim} in the field - zoom in to display labels.'

In [8]:
viewer1 = napari.Viewer()
ch1 = viewer1.add_image(ch0_list, name='ch1', colormap = 'green',blending='additive',contrast_limits=[0, 2048])
ch2 = viewer1.add_image(ch1_list, name='ch2', colormap = 'red',blending='additive',contrast_limits=[0, 2048])
viewer1.add_labels(np.zeros([ch0_list[0].shape[1],ch0_list[0].shape[2]]).astype(int),name='Labels')
viewer1.dims.events.current_step.connect(build_frame)
viewer1.camera.events.zoom.connect(build_frame)
viewer1.camera.events.center.connect(build_frame)
viewer1.layers['Labels'].events.visible.connect(build_frame)

<function __main__.build_frame()>

In [71]:
viewer1.layers['Labels'].corner_pixels

array([[5561, 5185],
       [5656, 5292]])

In [10]:
%%time
build_frame()

CPU times: total: 1.11 s
Wall time: 156 ms


## Test saving

In [11]:
from skimage.measure import regionprops

import sys
sys.path.append('..')
import tracks_interactions.db.db_functions as fdb

In [12]:
def trackDB_after_cellDB(session, cell_id):

    """
    Function to deal with tracks upon cell removal/adding
    cell_id - id of the removed cell
    current_frame
    """

    track = session.query(TrackDB).filter(TrackDB.track_id == cell_id).first()

    if track is not None:

        cells_t = session.query(CellDB.t).filter(CellDB.track_id == cell_id).all()
        cells_t  = [cell[0] for cell in cells_t]

        t_min = min(cells_t)
        t_max = max(cells_t)

        track.t_begin = t_min
        track.t_end = t_max

        session.commit()

In [24]:
def add_new_core_CellDB(session,current_frame,cell):
    
    """
    session
    current_frame 
    cell - regionprops format cell
    """

    # start the object
    cell_db = CellDB(id=cell.label, t=current_frame, track_id=cell.label)

    cell_db.row = int(cell.centroid[0])
    cell_db.col = int(cell.centroid[1])

    cell_db.bbox_0 = int(cell.bbox[0])
    cell_db.bbox_1 = int(cell.bbox[1])
    cell_db.bbox_2 = int(cell.bbox[2])
    cell_db.bbox_3 = int(cell.bbox[3])

    cell_db.mask = cell.image

    session.add(cell_db)
    session.commit()

    return cell_db

In [14]:
# function to add a complete cell
def add_new_CellDB(session, current_frame, cell, modified = True, ch_list = None, ch_names = None, ring_width = 5):
   
    """
    Function to add a complete cell
    """

    cell_db = add_new_core_CellDB(session,current_frame,cell) 

    # add signals to the cell
    new_signals = fdb.calculate_cell_signals(cell_db,ch_list=ch_list,ch_names=ch_names,ring_width=ring_width)
    cell_db.signals = new_signals

    # add modified tag to the cell
    tags = {}
    if modified:
        tags["modified"] = True
        cell_db.tags = tags

    session.commit()

    # deal with the tracks
    trackDB_after_cellDB(session, cell_db.track_id)

    

In [15]:
def remove_CellDB(session, cell_id, current_frame):

    cell = session.query(CellDB).filter(CellDB.track_id == cell_id).filter(CellDB.t==current_frame).first()
    
    session.delete(cell)
    session.commit()

    # deal with the tracks
    trackDB_after_cellDB(session, cell_id)     

In [65]:
corner_pixels = viewer1.layers["ch1"].corner_pixels * 2**viewer1.layers["ch1"].data_level
r_start = corner_pixels[0, 1]
r_stop = corner_pixels[1, 1]
c_start = corner_pixels[0, 2]
c_stop = corner_pixels[1, 2]

ind = viewer1.dims.current_step[0]
query_lim = 500

query = session.query(CellDB).filter(CellDB.t == ind).filter(CellDB.bbox_0 < int(r_stop)).filter(CellDB.bbox_1 < int(c_stop)).filter(CellDB.bbox_2 > int(r_start)).filter(CellDB.bbox_3 > int(c_start)).limit(query_lim).all()
query

[121010111 from frame 120 with track_id 20122 at (5099,4652),
 121010109 from frame 120 with track_id 20427 at (5062,4611),
 121010088 from frame 120 with track_id 20428 at (5058,4662),
 20834 from frame 120 with track_id 20834 at (5168,4666),
 121010138 from frame 120 with track_id 20841 at (5131,4702),
 121010136 from frame 120 with track_id 20843 at (5116,4736),
 121010139 from frame 120 with track_id 20847 at (5084,4746),
 121010140 from frame 120 with track_id 20851 at (5085,4699),
 121010110 from frame 120 with track_id 20855 at (5083,4629),
 121010240 from frame 120 with track_id 20856 at (5147,4580),
 121010229 from frame 120 with track_id 21275 at (5101,4517),
 121009951 from frame 120 with track_id 21736 at (5062,4566),
 121010311 from frame 120 with track_id 21738 at (5150,4536),
 121010239 from frame 120 with track_id 21739 at (5120,4595),
 121010402 from frame 120 with track_id 21744 at (5191,4576),
 121010525 from frame 120 with track_id 22267 at (5232,4517),
 121010310 f

In [66]:
%%time

current_frame = viewer1.dims.current_step[0]
ch_list=[ch0_list[0],ch1_list[0]]

refresh_status = False

# query 
query_ids = [cell.track_id for cell in query]

regionprops_results = regionprops(viewer1.layers['Labels'].data)

for cell_label in regionprops_results: 

    cell_label_id = cell_label.label

    if cell_label_id in query_ids:

        # remove from the list
        query_ids.remove(cell_label_id)

        # get the cell
        cell_query = [x for x in query if x.track_id == cell_label_id][0]
        
        # if modified
        row_diff = abs(cell_label.centroid[1] - cell_query.col) > 2
        col_diff = abs(cell_label.centroid[0] - cell_query.row) > 2
        mask_diff = not np.array_equal(cell_label.image, cell_query.mask)
        if (row_diff or col_diff or mask_diff): 

            # update the database
            print(f'{cell_label_id} has been modified')
            
            # remove old from the database
            remove_CellDB(session, cell_label_id, current_frame)

            # add new to the database
            add_new_CellDB(session, current_frame, cell_label, ch_list = ch_list)

            refresh_status = True
        
    else: 

        # a new cell
        print(f'{cell_label_id} has been added')
        # add new to the database
        add_new_CellDB(session, current_frame, cell_label, ch_list = ch_list)

        refresh_status = True

# for cells in query that are no longer in the field
for cell_id in query_ids:

    # cell is missing
    print(f'{cell_id} has been removed')
    # remove old from the database
    remove_CellDB(session, cell_id, current_frame)

    refresh_status = True

print(len(query_ids))

if refresh_status:
    
    print('building frame')
    # refresh labels layer
    build_frame()

20834 has been modified
0
building frame
CPU times: total: 1.23 s
Wall time: 316 ms


In [38]:
cell_label.centroid[1]

3960.1629213483147

In [ ]:
!= cell_query.col